In [2]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.models import Sequential, model_from_json
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from sklearn.preprocessing import LabelEncoder
import shutil

TRAIN_DIR = 'images/train'
TEST_DIR = 'images/test'

# Pehle sirf 3 emotions ke images filter karo
def filter_three_emotions(dir):
    """Sirf happy, neutral, sad ke images rakho"""
    valid_emotions = ['happy', 'neutral', 'sad']
    
    image_paths = []
    labels = []
    
    for label in os.listdir(dir):
        # Sirf valid emotions rakho
        if label in valid_emotions:
            label_path = os.path.join(dir, label)
            if os.path.isdir(label_path):
                for imagename in os.listdir(label_path):
                    image_paths.append(os.path.join(dir, label, imagename))
                    labels.append(label)
                print(f"✅ {label} completed")
        else:
            print(f"❌ Skipping {label} (not in required emotions)")
    
    return image_paths, labels

# Dataframe creation - Sirf 3 emotions
print("📁 Creating datasets with only 3 emotions...")
train = pd.DataFrame()
test = pd.DataFrame()

train['image'], train['label'] = filter_three_emotions(TRAIN_DIR)
test['image'], test['label'] = filter_three_emotions(TEST_DIR)

print(f"📊 Train dataset size: {len(train)} images")
print(f"📊 Test dataset size: {len(test)} images")

def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image, color_mode='grayscale', target_size=(48, 48))
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features), 48, 48, 1)
    return features

# Feature extraction
print("🔄 Extracting features...")
train_features = extract_features(train['image']) 
test_features = extract_features(test['image'])

# Normalization
x_train = train_features / 255.0
x_test = test_features / 255.0

# Label encoding - Sirf 3 classes
print("🔤 Encoding labels...")
le = LabelEncoder()
le.fit(train['label'])
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

# Sirf 3 classes
NUM_CLASSES = 3
y_train = to_categorical(y_train, num_classes=NUM_CLASSES)
y_test = to_categorical(y_test, num_classes=NUM_CLASSES)

print(f"✅ Classes: {list(le.classes_)}")

# Model architecture - Optimized for 3 classes
print("🤖 Creating model...")
model = Sequential()
model.add(Conv2D(64, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))

# OUTPUT LAYER: Sirf 3 neurons (happy, neutral, sad)
model.add(Dense(NUM_CLASSES, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

# Training - Epochs kam kardiya
print("🚀 Training model...")
history = model.fit(
    x=x_train, 
    y=y_train, 
    batch_size=64,  # Batch size adjust
    epochs=30,      # Epochs kam kardiya (30 is enough)
    validation_data=(x_test, y_test),
    verbose=1
)

# Model saving
model_json = model.to_json()
with open("emotiondetector.json", 'w') as json_file:
    json_file.write(model_json)
model.save("emotiondetector.h5")
print("✅ Model saved as emotiondetector.json and emotiondetector.h5")

# Labels with funny names
funny_labels = {
    'happy': 'Wah kia muskurahat hai! 😊',
    'neutral': 'Q dil nhi lag raha? 😐',
    'sad': 'Q chor kr chali gayi kia? 😢'
}

print("\n🎭 Emotion Labels:")
for emotion, funny_text in funny_labels.items():
    print(f"  {emotion}: {funny_text}")

# Model loading test
with open("emotiondetector.json", "r") as json_file:
    loaded_model_json = json_file.read()

loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("emotiondetector.h5")
print("✅ Model loaded successfully!")

# Performance check
train_acc = history.history['accuracy'][-1] * 100
val_acc = history.history['val_accuracy'][-1] * 100

print(f"\n📊 Final Results:")
print(f"  Training Accuracy: {train_acc:.2f}%")
print(f"  Validation Accuracy: {val_acc:.2f}%")
print(f"  Trained for {len(history.history['accuracy'])} epochs")

📁 Creating datasets with only 3 emotions...
✅ happy completed
✅ neutral completed
✅ sad completed
✅ happy completed
✅ neutral completed
✅ sad completed
📊 Train dataset size: 17084 images
📊 Test dataset size: 4180 images
🔄 Extracting features...


  0%|          | 0/17084 [00:00<?, ?it/s]

  0%|          | 0/4180 [00:00<?, ?it/s]

🔤 Encoding labels...
✅ Classes: ['happy', 'neutral', 'sad']
🤖 Creating model...


c:\Users\afroz\miniconda3\envs\my_env\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

 Total params: 1,451,779 (5.54 MB)

 Trainable params: 1,451,779 (5.54 MB)

 Non-trainable params: 0 (0.00 B)

🚀 Training model...
Epoch 1/30
267/267 ━━━━━━━━━━━━━━━━━━━━ 56s 192ms/step - accuracy: 0.4185 - loss: 1.0832 - val_accuracy: 0.4459 - val_loss: 1.0622
Epoch 2/30
267/267 ━━━━━━━━━━━━━━━━━━━━ 48s 181ms/step - accuracy: 0.4764 - loss: 1.0170 - val_accuracy: 0.5897 - val_loss: 0.8832
Epoch 3/30
267/267 ━━━━━━━━━━━━━━━━━━━━ 47s 177ms/step - accuracy: 0.5859 - loss: 0.8695 - val_accuracy: 0.6443 - val_loss: 0.7706
Epoch 4/30
267/267 ━━━━━━━━━━━━━━━━━━━━ 48s 179ms/step - accuracy: 0.6223 - loss: 0.8054 - val_accuracy: 0.6699 - val_loss: 0.7278
Epoch 5/30
267/267 ━━━━━━━━━━━━━━━━━━━━ 48s 181ms/step - accuracy: 0.6467 - loss: 0.7666 - val_accuracy: 0.6856 - val_loss: 0.6854
Epoch 6/30
267/267 ━━━━━━━━━━━━━━━━━━━━ 48s 178ms/step - accuracy: 0.6606 - loss: 0.7347 - val_accuracy: 0.6959 - val_loss: 0.6753
Epoch 7/30
267/267 ━━━━━━━━━━━━━━━━━━━━ 47s 174ms/step - accuracy: 0.6735 - loss: 0.7178 - val_accuracy: 0.7089 - val_loss: 0.6537
Epoch 8/30
267/267 ━━━━━━━━━━━━━━━━━━━━ 45s 169ms/step - accura

✅ Model saved as emotiondetector.json and emotiondetector.h5

🎭 Emotion Labels:
  happy: Wah kia muskurahat hai! 😊
  neutral: Q dil nhi lag raha? 😐
  sad: Q chor kr chali gayi kia? 😢
✅ Model loaded successfully!

📊 Final Results:
  Training Accuracy: 79.76%
  Validation Accuracy: 75.10%
  Trained for 30 epochs
